## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.user_based_CF import UserBasedCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9776855    ┆ 1994544 ┆ 80.0           ┆ 100.0      │
│ 9788677    ┆ 78997   ┆ 50700.0        ┆ 100.0      │
│ 9785349    ┆ 2059115 ┆ 122.0          ┆ 100.0      │
│ 9789065    ┆ 1689641 ┆ 1785.0         ┆ 100.0      │
│ 9771916    ┆ 484479  ┆ 3.6977e11      ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [ ]:
recommender = UserBasedCollaborativeRecommender(train_df)
recommender.fit(scroll_weight=0.1,readtime_weight=1)

{1994544: [(210662, np.float64(0.33036520462829333)),
  (2501933, np.float64(0.33036520462829333)),
  (620638, np.float64(0.33036520462829333)),
  (798921, np.float64(0.33036520462829333)),
  (2354225, np.float64(0.3240281454933236)),
  (1033227, np.float64(0.3169993135810234)),
  (2312861, np.float64(0.2968859053319317)),
  (453011, np.float64(0.26537987184429634)),
  (115886, np.float64(0.25521775464846697)),
  (390747, np.float64(0.25521775464846697))],
 78997: [(2133038, np.float64(0.9999695709970239)),
  (2213436, np.float64(0.9987462680386716)),
  (73852, np.float64(0.6302295156551849)),
  (2022706, np.float64(0.39670504343934454)),
  (2143164, np.float64(0.3279485897433859)),
  (338173, np.float64(0.29198239834731887)),
  (2521895, np.float64(0.1859533243229048)),
  (600400, np.float64(0.13485966766002144)),
  (308179, np.float64(0.101936571959239)),
  (998348, np.float64(0.06929906248354278))],
 2059115: [(221100, np.float64(0.9791677422537386)),
  (2061500, np.float64(0.979167

This model treats all interactions equally, no matter how long an interaction lasted

In [4]:
binary_recommender = UserBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{1994544: [(453011, np.float64(0.282842712474619)),
  (2312861, np.float64(0.282842712474619)),
  (2230601, np.float64(0.282842712474619)),
  (2307854, np.float64(0.23094010767585038)),
  (515977, np.float64(0.22677868380553634)),
  (2024834, np.float64(0.22677868380553634)),
  (2363417, np.float64(0.21213203435596428)),
  (481703, np.float64(0.21213203435596428)),
  (684820, np.float64(0.19999999999999996)),
  (1111620, np.float64(0.19999999999999996))],
 78997: [(858439, np.float64(0.20851441405707472)),
  (715131, np.float64(0.20851441405707472)),
  (237447, np.float64(0.20851441405707472)),
  (2418074, np.float64(0.20064308847628198)),
  (1675585, np.float64(0.20064308847628198)),
  (549932, np.float64(0.18200630207731605)),
  (873071, np.float64(0.18057877962865376)),
  (2573518, np.float64(0.18057877962865376)),
  (1068948, np.float64(0.17201561551404665)),
  (800928, np.float64(0.17025130615174966))],
 2059115: [(1788303, np.float64(0.22645540682891907)),
  (2531022, np.float64(

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [5]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9774404, 9790559, 9772453, 9771554]
reccomended for user  620796 :  [9771224, 9778942, 9776152, 9771166, 9771151]
reccomended for user  1067393 :  [9771916, 9784506, 9782722, 9789446, 9777648]
reccomended for user  1726258 :  [9771224, 9770082, 9774079, 9783379, 9769197]
reccomended for user  17205 :  [9777529, 9783657, 9774187, 9788666, 9769800]


In [6]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9769504, 9776855, 9759955, 9787465, 9781902]
reccomended for user  620796 :  [9771168, 9771576, 9770082, 9771686, 9783278]
reccomended for user  1067393 :  [9787098, 9782746, 9775277, 9775621, 9782282]
reccomended for user  1726258 :  [9771224, 9771113, 9773846, 9757574, 9774789]
reccomended for user  17205 :  [9775562, 9771846, 9779674, 9779269, 9777804]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [7]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.002), 'NDCG@K': np.float64(0.020167768749982218)}

The binary reccomender model only reccomending articles the user has not yet read

In [8]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0008163265306122449),
 'NDCG@K': np.float64(0.004831825400879857)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [9]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0015238095238095239),
 'NDCG@K': np.float64(0.018823356598396773)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [10]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0007017543859649123),
 'NDCG@K': np.float64(0.006370440394136899)}

## Model Experimentation

In [11]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9787524, 9778448, 9783824, 9756075, 9773248, 9782722, 9772355, 9771460, 9538375, 9788362, 9772508, 9790942, 9786718, 9787487, 9780193, 9776862, 9783655, 9738729, 9771627, 9780849, 9673979, 9786111}
[9778351, 9774404, 9790559, 9772453, 9771554, 9776497, 9776147, 9778945, 9781389, 9771919, 9771333, 9772813, 9772300, 9767830, 9782869, 9771168, 9772903, 9783657, 9774972, 9774229, 9777811, 9783993, 9765759, 9778375, 9773846, 9774120, 9780280, 9786378, 9779269, 9771330, 9790784, 9777406, 9780284, 9779408, 9777621, 9772343, 9769917, 9784425, 9780677, 9780384, 9775647, 9781785, 9779650, 9776560, 9790987, 9772963, 9779737, 9784863, 9779724, 9778915, 9769624, 9776985, 9780406, 9775881, 9785668, 9772227, 9780960, 9465878, 9773574, 9771916, 9771796, 9779045, 9771350, 9771903, 9784696, 9779133, 9783655, 9773137, 9773210, 9784852, 9779294, 9771187, 9777299, 9784710, 9772442, 9776855, 9780496, 9774187, 9770327, 9771170, 9784444, 9781423]
Yes


In [12]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9787524, 9778448, 9783824, 9756075, 9773248, 9782722, 9772355, 9771460, 9538375, 9788362, 9772508, 9790942, 9786718, 9787487, 9780193, 9776862, 9783655, 9738729, 9771627, 9780849, 9673979, 9786111}
[9778351, 9774404, 9790559, 9772453, 9771554, 9776497, 9776147, 9778945, 9781389, 9771919, 9771333, 9772813, 9772300, 9767830, 9782869, 9771168, 9772903, 9783657, 9774972, 9774229, 9777811, 9783993, 9765759, 9778375, 9773846, 9774120, 9780280, 9786378, 9771330, 9779269, 9790784, 9777406, 9780284, 9779408, 9777621, 9772343, 9769917, 9780677, 9784425, 9780384, 9775647, 9781785, 9779650, 9776560, 9790987, 9772963, 9784863, 9779737, 9779724, 9776985, 9769624, 9778915, 9780406, 9775881, 9772227, 9785668, 9780960, 9465878, 9773574, 9771916, 9771796, 9779045, 9771350, 9771903, 9784696, 9779133, 9783655, 9773137, 9773210, 9784852, 9771187, 9777299, 9779294, 9784710, 9772442, 9776855, 9774187, 9780496, 9770327, 9771170, 9784444, 9781423]
Yes


In [13]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

{'precision@k': np.float64(0.003590255022082918),
 'recall@k': np.float64(0.006766059250927679),
 'fpr@k': np.float64(0.0018853695502671316)}

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [14]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 20:20:53] [setup] RAM Tracking...
[codecarbon INFO @ 20:20:53] [setup] CPU Tracking...
[codecarbon WARNING @ 20:20:53] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon INFO @ 20:20:54] CPU Model on constant consumption mode: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 20:20:54] [setup] GPU Tracking...
[codecarbon INFO @ 20:20:55] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:20:55] >>> Tracker's metadata:
[codecarbon INFO @ 20:20:55]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 20:20:55]   Python version: 3.11.1
[codecarbon INFO @ 20:20:55]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 20:20:55]   Available RAM : 7.338 GB
[codecarbon INFO @ 20:20:55]   CPU count: 12
[codecarbon INFO @ 20:20:55]   CPU model: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 20:20:55]   GPU count: 1
[codecarbon INFO @ 20:20:55]   GPU model: 1 x NVIDIA GeForce GTX 1650
[codecarbon INFO @ 20:20:56] Saving emissions data to file c:\Users\chris\Desktop\NTNU Ting\8. Semester\Anbefalingssystemer\Project\TDT4215\recommender_system\demostrations\output\user_based_fit_emission.csv
[codecarbon INFO @ 20:21:11] Energy c

{'fit': ({1994544: [(210662, np.float64(0.33036520462829333)),
    (2501933, np.float64(0.33036520462829333)),
    (620638, np.float64(0.33036520462829333)),
    (798921, np.float64(0.33036520462829333)),
    (2354225, np.float64(0.3240281454933236)),
    (1033227, np.float64(0.3169993135810234)),
    (2312861, np.float64(0.2968859053319317)),
    (453011, np.float64(0.26537987184429634)),
    (115886, np.float64(0.25521775464846697)),
    (390747, np.float64(0.25521775464846697))],
   78997: [(2133038, np.float64(0.9999695709970239)),
    (2213436, np.float64(0.9987462680386716)),
    (73852, np.float64(0.6302295156551849)),
    (2022706, np.float64(0.39670504343934454)),
    (2143164, np.float64(0.3279485897433859)),
    (338173, np.float64(0.29198239834731887)),
    (2521895, np.float64(0.1859533243229048)),
    (600400, np.float64(0.13485966766002144)),
    (308179, np.float64(0.101936571959239)),
    (998348, np.float64(0.06929906248354278))],
   2059115: [(221100, np.float64(0.97